# Языковые модели

Языковые модели играют важную роль в системах распознавания речи, помогая создавать более грамотные и лексически корректные тексты. В данной работе мы будем изучать нграмные языковые модели, которые позволяют довольно легко оценить вероятность и правдоподобность текста.

В нграмной языковой модели, нграм - это последовательность из n слов в тексте. Например, в предложении "по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас", биграмами будут "по-моему мы", "мы сэкономим", "сэкономим уйму" итд. Языковые модели оценивают вероятность появления последовательности слов, исходя из статистики появления каждого из нграм в обучающей выборке.

Порядком (order) нграм языковой модели называют максимальную длину нграм, которую учитывает модель.

Практическая работа разделена на 2 части:
1. Построение нграмой языковой модели - основная часть, 10 баллов
1. Предсказание с помощью языковой модели - дополнительная часть, 6 балла



Полезные сслыки:
* arpa формат - https://cmusphinx.github.io/wiki/arpaformat/
* обучающие материалы - https://pages.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture13/lecture13_ngrams_with_SRILM.pdf
* обучающие материалы.2 - https://cjlise.github.io/machine-learning/N-Gram-Language-Model/

In [1]:
import numpy as np
from collections import defaultdict, Counter
from typing import List, Dict, Tuple

# 1. Построение нграмной языковой модели. (10 баллов)


Вероятность текста с помощью нграмной языковой модели можно вычислить по формуле:
$$ P(w_1, w_2, .., w_n) = {\prod{{P_{i=0}^{n}(w_i| w_{i-order}, .., w_{i-1})}}} $$

В простом виде, при обучении нграмной языковой модели, чтобы рассчитать условную вероятность каждой нграмы, используется формула, основанная на количестве появлений нграмы в обучающей выборке. Формула выглядит следующим образом:
$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i})} \over {count(w_{i-order},..., w_{i-1})}} $$

Поскольку униграмы не содержат в себе какого-дибо контекста, вероятность униграмы можно посчитать поделив кол-во этой слова на общее количество слов в обучающей выборке.


In [41]:
# в первую очередь нам понадобится подсчитать статистику по обучающей выборке
def count_ngrams(train_text: List[str], order=3, bos=True, eos=True) -> Dict[Tuple[str], int]:
    ngrams = defaultdict(int)

    for sentence in train_text:
        # Разбиваем предложение на слова
        words = sentence.split()

        # добавляем при необходимости символ начала строки и/или конца строки
        if bos:
          words = ['<s>'] + words
        if eos:
          words = words + ['</s>']

        # итерируемся по всем n-граммам
        for n in range(1, order + 1):
            # Генерация n-грамм
            current_ngrams = zip(*[words[i:] for i in range(n)])

            # Суммируем уже существующие данные и данные для текущего предложения
            ngrams = dict(Counter(ngrams) + Counter(current_ngrams))

    return dict(ngrams)

In [3]:
def test_count_ngrams():
    assert count_ngrams(['привет привет как дела'], order=1, bos=True, eos=True) == {
        ('<s>',): 1,
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1,
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=True) == {
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1,
        ('</s>',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=1, bos=False, eos=False) == {
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1
    }
    assert count_ngrams(['привет привет как дела'], order=2, bos=False, eos=False) == {
        ('привет',): 2,
        ('как',): 1,
        ('дела',): 1,
        ('привет', 'привет'): 1,
        ('привет', 'как'): 1,
        ('как', 'дела'): 1
    }
    assert count_ngrams(['привет ' * 6], order=2, bos=False, eos=False) == {
        ('привет',): 6,
        ('привет', 'привет'): 5
    }
    result = count_ngrams(['практическое сентября',
                           'второе практическое занятие пройдет в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=5)
    assert result[('<s>',)] == 3
    assert result[('32',)] == 3
    assert result[('<s>', 'в', 'офлайне', 'в', '32')] == 1
    assert result[('офлайне', 'в', '32', '12', '</s>')] == 1
    print('Test 1a passed')


test_count_ngrams()

Test 1a passed



Простой подход к вычислению вероятностей через количество нграм имеет существенный недостаток. Если в тексте встретится нграмма, которой не было в обучающей выборке, то вероятность всего текста будет равна нулю.

Чтобы избежать данного недостатка, вводится специальное сглаживание - add-k сглаживание ([Additive, Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)). Данная техника позволяет учитывать нграмы, не встретившиеся в обучающей выборке, и при этом не делает вероятность текста равной нулю.

Формула сглаживания Лапласа выглядит следующим образом:

$$ P(w_i| w_{i-order}, .., w_{i-1}) = {{count(w_{i-order}, .., w_{i}) + k} \over {count(w_{i-order},..., w_{i-1}) + k*V}} $$

Здесь V - количество слов в словаре, а k - гиперпараметр, который контролирует меру сглаживания. Как правило, значение k выбирается экспериментально, чтобы найти оптимальный баланс между учетом редких нграм и сохранением вероятности для часто встречающихся нграм.


In [99]:
def calculate_ngram_prob(ngram: Tuple[str], counts: Dict[Tuple[str], int], V=None, k=0) -> float:
    # Получаем частоту n-граммы
    ngram_count = counts.get(ngram, 0)
    context_count = 0
    n = len(ngram)

    # Если n > 1, определяем контекст
    if n > 1:
        context = ngram[:-1]  # (n-1)-грамма
        context_count = counts.get(context, 0)
    # Если n == 1, то мы хотим посчитать частоты всех однословных n-грамм
    else:
        for ngram_text, count in counts.items():
            # Учитываем только одномерные n-граммы
            if len(ngram_text) == 1:
                context_count += count

    # Если размер словаря V не указан, вычисляем его через уникальные слова в counts
    V = len(set(word for ngram in counts.keys() for word in ngram)) if V is None else V

    # Лапласовское сглаживание
    prob = (ngram_count + k) / (context_count + k * V)

    return prob

In [100]:
def test_calculate_ngram_prob():
    counts = count_ngrams(['практическое сентября',
                           'второе практическое занятие в офлайне 32 сентября в 12 часов 32 минуты',
                           'в офлайне в 32 12'], order=4)
    assert calculate_ngram_prob(('в', 'офлайне'), counts) == 0.5
    assert calculate_ngram_prob(('в', ), counts) == 4/25
    assert calculate_ngram_prob(('в', ), counts, k=0.5) == (4+0.5)/(25+0.5*12)
    assert calculate_ngram_prob(('в', 'офлайне', 'в', '32'), counts) == 1.0
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=1) == 0.1875
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=0) == 0.0
    assert calculate_ngram_prob(('в', 'онлайне'), counts, k=1) == 0.0625
    assert calculate_ngram_prob(('в', 'офлайне'), counts, k=0.5) == 0.25

    print("Test 1.b passed")


test_calculate_ngram_prob()

Test 1.b passed


Основной метрикой язковых моделей является перплексия.

Перплексия  — безразмерная величина, мера того, насколько хорошо распределение вероятностей предсказывает выборку. Низкий показатель перплексии указывает на то, что распределение вероятности хорошо предсказывает выборку.

$$ ppl = {P(w_1, w_2 ,..., w_N)^{- {1} \over {N}}} $$


In [219]:
# Языковая модель
class NgramLM:
    def __init__(self, order=3, bos=True, eos=True, k=1, predefined_vocab=None):
        self.order = order
        self.eos = eos
        self.bos = bos
        self.k = k
        self.vocab = predefined_vocab
        self.ngrams_count = None

    @property
    def V(self) -> int:
        return len(self.vocab)

    def fit(self, train_text: List[str]) -> None:
        # гиспользуем уже реализованную функцию для генерации всех n-грамм и их
        # количества
        self.ngrams_count = count_ngrams(train_text, order=self.order, bos=self.bos, eos=self.eos)
        # обновляем словарь
        self.vocab = [ngram for ngram in self.ngrams_count.keys() if len(ngram) == 1]

    def predict_ngram_log_proba(self, ngram: Tuple[str]) -> float:
        # считаем вероятность n-граммы
        proba = calculate_ngram_prob(ngram, self.ngrams_count, self.V, self.k)
        # берем логарифм
        return np.log(proba)

    def predict_log_proba(self, words: List[str]) -> float:
        # если нужно, то добавляем символ начала и/или конца строки
        if self.bos:
            words = ['<s>'] + words
        if self.eos:
            words = words + ['</s>']

        logprob = 0
        # удаляем пустые строки, чтобы не обрабатывать их в дальнейшем
        words = [word for word in words if word.strip() != ""]

        # применяем chain rule, чтобы посчитать логарифм вероятности всей строки
        start = 1 - self.order
        used_ngrams_count = 0

        for i in range(len(words)):
          # Определяем начальный индекс для текущей n-граммы
          actual_start = max(0, i + 1 - self.order)

          # Берём n-грамму, начиная с actual_start и заканчивая текущей позицией (i + 1)
          ngram = tuple(words[actual_start:i + 1])

          # Считаем логарифм вероятности для текущей n-граммы
          logprob += self.predict_ngram_log_proba(ngram)

          # Увеличиваем счётчик использованных n-грамм
          used_ngrams_count += 1

        return logprob

    def ppl(self, text: List[str]) -> float:
        # подсчет перплексии
        # Для того, чтобы ваш код был численно стабильным,
        # не считайте формулу напрямую, а воспользуйтесь переходом к логарифмам вероятностей
        sum_proba = 0
        n = 0

        # итерируемся по всем строкам в тексте
        for line in text:
            # разбиваем на слова
            words = line.split()
            # считаем вероятность
            log_proba = self.predict_log_proba(words)
            # суммируем вероятности
            sum_proba += log_proba
            # суммируем количество n-грамм в текущем предложении
            n += max(1, len(words) + 1 + self.eos)

        perplexity = np.exp(-sum_proba / n)

        return perplexity

In [246]:
def test_lm():
    train_data = ["по-моему мы сэкономим уйму времени если я сойду с ума прямо сейчас",
                  "если я сойду с ума прямо сейчас по-моему мы сэкономим уйму времени",
                  "мы сэкономим уйму времени если я сейчас сойду с ума по-моему"]
    global lm
    lm = NgramLM(order=2)
    lm.fit(train_data)
    assert lm.V == 14
    assert np.isclose(lm.predict_log_proba(['мы']), lm.predict_log_proba(["если"]))
    assert lm.predict_log_proba(["по-моему"]) > lm.predict_log_proba(["если"])

    gt = ((3+1)/(41 + 14) * 1/(3+14))**(-1/2)
    ppl = lm.ppl([''])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"

    gt = ((3+1)/(41 + 14) * 1/(3+14) * 1/(14)) ** (-1/3)
    ppl = lm.ppl(['ЧТО'])
    assert  np.isclose(ppl, gt), f"{ppl=} {gt=}"

    test_data = ["по-моему если я прямо сейчас сойду с ума мы сэкономим уйму времени"]
    ppl = lm.ppl(test_data)
    assert round(ppl, 2) == 7.33, f"{ppl}"
test_lm()

# 2. Предсказания с помощью языковой модели (6 балла)

In [261]:
def predict_next_word(lm: NgramLM, prefix: List[str], topk=4):
    # TODO реализуйте функцию, которая предсказывает продолжение фразы.
    # верните topk наиболее вероятных продолжений фразы prefix
    probabilities = []

    for word in lm.vocab:
        ngram = tuple(prefix + list(word))
        proba = lm.predict_ngram_log_proba(ngram)

        probabilities.append({
            "ngram": ngram[-1],
            "proba": proba,
        })

    probabilities.sort(key=lambda row: row["proba"], reverse=True)
    probabilities = probabilities[:topk]

    next_words = [(row["ngram"], row["proba"]) for row in probabilities]
    return next_words

In [262]:
predict_next_word(lm, ["ума"])

[('прямо', -1.7346010553881064),
 ('по-моему', -2.1400661634962708),
 ('<s>', -2.833213344056216),
 ('мы', -2.833213344056216)]

Попробуйте обучить ngram языковую модель на нескольких стихотворениях. Не забудьте трансформировать стихотворение в удобный для ngram модели формат (как сделать так, чтобы модель моделировала рифму?).
Попробуйте сгенерировать продолжение для стихотворения с помощью такой языковой модели.

In [167]:
text = """
    Как-то в полночь, в час угрюмый, полный тягостною думой,
    Над старинными томами я склонялся в полусне,
    Грезам странным отдавался, — вдруг неясный звук раздался,
    Будто кто-то постучался — постучался в дверь ко мне.
    «Это, верно, — прошептал я, — гость в полночной тишине,
    Гость стучится в дверь ко мне».
    Ясно помню… Ожиданье… Поздней осени рыданья…
    И в камине очертанья тускло тлеющих углей…
    О, как жаждал я рассвета, как я тщетно ждал ответа
    На страданье без привета, на вопрос о ней, о ней —
    О Леноре, что блистала ярче всех земных огней, —
    О светиле прежних дней.
    И завес пурпурных трепет издавал как будто лепет,
    Трепет, лепет, наполнявший темным чувством сердце мне.
    Непонятный страх смиряя, встал я с места, повторяя:
    «Это только гость, блуждая, постучался в дверь ко мне,
    Поздний гость приюта просит в полуночной тишине —
    Гость стучится в дверь ко мне».
    «Подавив свои сомненья, победивши опасенья,
    Я сказал: «Не осудите замедленья моего!
    Этой полночью ненастной я вздремнул, — и стук неясный
    Слишком тих был, стук неясный, — и не слышал я его,
    Я не слышал…» — тут раскрыл я дверь жилища моего:
    Тьма — и больше ничего.
    Взор застыл, во тьме стесненный, и стоял я изумленный,
    Снам отдавшись, недоступным на земле ни для кого;
    Но как прежде ночь молчала, тьма душе не отвечала,
    Лишь — «Ленора!» — прозвучало имя солнца моего, —
    Это я шепнул, и эхо повторило вновь его, —
    Эхо, больше ничего.
    Вновь я в комнату вернулся — обернулся — содрогнулся, —
    Стук раздался, но слышнее, чем звучал он до того.
    «Верно, что-нибудь сломилось, что-нибудь пошевелилось,
    Там, за ставнями, забилось у окошка моего,
    Это — ветер, — усмирю я трепет сердца моего, —
    Ветер — больше ничего».
    Я толкнул окно с решеткой, — тотчас важною походкой
    Из-за ставней вышел Ворон, гордый Ворон старых дней,
    Не склонился он учтиво, но, как лорд, вошел спесиво
    И, взмахнув крылом лениво, в пышной важности своей
    Он взлетел на бюст Паллады, что над дверью был моей,
    Он взлетел — и сел над ней.
    От печали я очнулся и невольно усмехнулся,
    Видя важность этой птицы, жившей долгие года.
    «Твой хохол ощипан славно, и глядишь ты презабавно, —
    Я промолвил, — но скажи мне: в царстве тьмы, где ночь всегда,
    Как ты звался, гордый Ворон, там, где ночь царит всегда?»
    Молвил Ворон: «Никогда».
"""

In [140]:
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [155]:
def preprocess(text):
    # Приводим к нижнему регистру
    normalized = text.strip().lower()

    # Удаляем все, кроме букв, цифр, тире, пробелов и разделителей предложений
    normalized = re.sub(r"[^\w\d\.\?!\-\s]", " ", normalized)

    # Удаляем тире между словами и лишние пробелы
    normalized = re.sub(r"\s*-\s*", " ", normalized)
    normalized = re.sub(r"\s+", " ", normalized).strip()

    # Используем nltk для разбивки на предложения
    sentences = nltk.tokenize.sent_tokenize(normalized)

    # Удаляем точки в конце предложений и другие знаки препинания
    sentences = [re.sub(r"[\.!?]", "", sentence).strip() for sentence in sentences]

    return sentences

In [149]:
sentences = preprocess(text)

In [150]:
sentences

['как то в полночь в час угрюмый полный тягостною думой над старинными томами я склонялся в полусне грезам странным отдавался вдруг неясный звук раздался будто кто то постучался постучался в дверь ко мне',
 'это верно прошептал я гость в полночной тишине гость стучится в дверь ко мне',
 'ясно помню ожиданье поздней осени рыданья и в камине очертанья тускло тлеющих углей о как жаждал я рассвета как я тщетно ждал ответа на страданье без привета на вопрос о ней о ней о леноре что блистала ярче всех земных огней о светиле прежних дней',
 'и завес пурпурных трепет издавал как будто лепет трепет лепет наполнявший темным чувством сердце мне',
 'непонятный страх смиряя встал я с места повторяя это только гость блуждая постучался в дверь ко мне поздний гость приюта просит в полуночной тишине гость стучится в дверь ко мне',
 'подавив свои сомненья победивши опасенья я сказал не осудите замедленья моего',
 'этой полночью ненастной я вздремнул и стук неясный слишком тих был стук неясный и не слыша

In [242]:
lm = NgramLM(order=2, bos=False, eos=False)
lm.fit(sentences)

In [243]:
first_word = "ворон"
length = 10

answer = first_word
next_word = first_word

for _ in range(0, length):
    next_word = predict_next_word(lm, [next_word], topk=1)[0][0]
    answer += " " + next_word

answer

'ворон там за ставнями забилось у окошка моего это я в'

### Попробуем добавить разметку рифмы

Для этого будет разбивать по строкам, а не по предложениям. Попробуем использовать также библиотеку pronouncing для определения наличия рифмы.


In [ ]:
!pip install pronouncing
import pronouncing

In [205]:
def get_last_word(line: str) -> str:
    # Получаем последнее слово в строке
    words = line.split()
    return words[-1] if words else ""

def add_rhyme_labels(text: list) -> list:
    # Добавляем метки рифмы к строкам
    rhyme_groups = defaultdict(list)

    # Группируем строки по рифмам их последних слов
    for idx, line in enumerate(text):
        last_word = get_last_word(line)
        # Получаем рифмы для последнего слова
        rhymes = pronouncing.rhymes(last_word)

        # Если рифмы найдены, используем их для создания меток
        if rhymes:
            # Добавляем текущую строку в группу рифм
            rhyme_groups[last_word].extend([idx] + [text.index(rhyme) for rhyme in rhymes if rhyme in text])
        else:
            # Если рифм нет, используем последние 2 буквы как признак рифмы
            rhyme_groups[last_word[-2:]].append(idx)

    labeled_lines = text[:]
    rhyme_label = 1
    for group in rhyme_groups.values():
        label = f"_rhyme{rhyme_label}"
        for idx in group:
            labeled_lines[idx] += label
        rhyme_label += 1

    return labeled_lines

In [206]:
lines = text.split('\n')

normalized_lines = []
for line in lines:
  normalized = line.strip().lower()
  # Удаляем все, кроме букв, цифр, тире, пробелов и разделителей предложений
  normalized = re.sub(r"[^\w\d\.\?!\-\s]", " ", normalized)

  # Удаляем тире между словами и лишние пробелы
  normalized = re.sub(r"\s*-\s*", " ", normalized)
  normalized = re.sub(r"\s+", " ", normalized).strip()

  # Удаляем точки в конце предложений и другие знаки препинания
  normalized = re.sub(r"[\.!?]", "", normalized).strip()
  normalized_lines.append(normalized)

In [207]:
normalized_lines = list(filter(None, normalized_lines))

In [208]:
normalized_lines

['как то в полночь в час угрюмый полный тягостною думой',
 'над старинными томами я склонялся в полусне',
 'грезам странным отдавался вдруг неясный звук раздался',
 'будто кто то постучался постучался в дверь ко мне',
 'это верно прошептал я гость в полночной тишине',
 'гость стучится в дверь ко мне',
 'ясно помню ожиданье поздней осени рыданья',
 'и в камине очертанья тускло тлеющих углей',
 'о как жаждал я рассвета как я тщетно ждал ответа',
 'на страданье без привета на вопрос о ней о ней',
 'о леноре что блистала ярче всех земных огней',
 'о светиле прежних дней',
 'и завес пурпурных трепет издавал как будто лепет',
 'трепет лепет наполнявший темным чувством сердце мне',
 'непонятный страх смиряя встал я с места повторяя',
 'это только гость блуждая постучался в дверь ко мне',
 'поздний гость приюта просит в полуночной тишине',
 'гость стучится в дверь ко мне',
 'подавив свои сомненья победивши опасенья',
 'я сказал не осудите замедленья моего',
 'этой полночью ненастной я вздремну

In [209]:
rhymed = add_rhyme_labels(normalized_lines)

In [210]:
rhymed

['как то в полночь в час угрюмый полный тягостною думой_rhyme1',
 'над старинными томами я склонялся в полусне_rhyme2',
 'грезам странным отдавался вдруг неясный звук раздался_rhyme3',
 'будто кто то постучался постучался в дверь ко мне_rhyme2',
 'это верно прошептал я гость в полночной тишине_rhyme2',
 'гость стучится в дверь ко мне_rhyme2',
 'ясно помню ожиданье поздней осени рыданья_rhyme4',
 'и в камине очертанья тускло тлеющих углей_rhyme5',
 'о как жаждал я рассвета как я тщетно ждал ответа_rhyme6',
 'на страданье без привета на вопрос о ней о ней_rhyme5',
 'о леноре что блистала ярче всех земных огней_rhyme5',
 'о светиле прежних дней_rhyme5',
 'и завес пурпурных трепет издавал как будто лепет_rhyme7',
 'трепет лепет наполнявший темным чувством сердце мне_rhyme2',
 'непонятный страх смиряя встал я с места повторяя_rhyme8',
 'это только гость блуждая постучался в дверь ко мне_rhyme2',
 'поздний гость приюта просит в полуночной тишине_rhyme2',
 'гость стучится в дверь ко мне_rhyme

In [238]:
lm_r = NgramLM(order=2, eos=False, bos=False)
lm_r.fit(rhymed)

In [240]:
first_word = "ворон"
length = 10

answer = first_word
next_word = first_word

for _ in range(0, length):
    next_word = predict_next_word(lm_r, [next_word], topk=1)[0][0]
    answer += " " + next_word

answer

'ворон там за ставнями забилось у окошка моего_rhyme9 как то в'